In [95]:
import pandas as pd
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="Sales_by_product_log.log",filemode="w")

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv',parse_dates=['payment_date'])
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv',parse_dates=['created_at'])
venue=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\cafe.csv',parse_dates=['last_update'])
address=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\address.csv',parse_dates=['last_update'])
product=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\item.csv',parse_dates=['last_update'])

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=False)
merged_df=merged_df.merge(venue,on='cafe_id',how='left',indicator=False)
merged_df=merged_df.merge(address,on='address_id',how='left',indicator=False)
merged_df=merged_df.merge(product,on='item_id',how='left',indicator=False)


merged_df=merged_df.drop(columns=['payment_id','customer_id_x','emp_id','payment_type',
                        'order_id','item_id','customer_id_y','delivery',
                        'delivery_address_id','created_at','customer_modifications','delivery_notes',
                        'address_id','address_first_line','postal_code','last_update_y',
                        'last_update','recipe_id','address_second_line',
                        'manager_emp_id','phone_number','last_update_x','amount'])

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte
merged_df['Year']=payment_date_converted.year
merged_df['Year']=merged_df['Year'].astype('str')

merged_df['Month']=payment_date_converted.month 
merged_df['Month']=merged_df['Month'].astype('str')
merged_df['Month'] = merged_df['Month'].str.zfill(2)

merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

merged_df=merged_df.drop(columns=['payment_date'])
venue_dictionary={1:"New York",2:"Miami",3:"Lima",4:"Buenos Aires",5:"Seattle"}
merged_df['cafe_id']=merged_df['cafe_id'].map(venue_dictionary)
region_dictionary={"Peru":"LATAM","Argentina":"LATAM","USA":"North America"}
merged_df['region']=merged_df['country'].map(region_dictionary)
product_category_dictionary={"butter croissant":"food","cinnamon bun":"food", "multigrain toast with butter":"food",
                             "iced coffee":"drink","hot coffee":"drink"}
merged_df['product_category']=merged_df['item_name'].map(product_category_dictionary)
merged_df.rename(columns={'cafe_id':'venue','item_name':'product_name',
                          'item_size':	'product_size','item_price':'product_price',
                          'item_quantity':'qty_sold',
                          'Year':'year','Year_Month':'year_month','Month':'month' },inplace=True)


merged_df=merged_df.loc[:,['region','country','city','venue','year','year_month','month',
                           'product_category','product_name','product_size',
                           'product_price','qty_sold']]
#TO DO:substitute blanks in "product_size" with "one size"

merged_df['product_size']=merged_df['product_size'].fillna("one size")

merged_df['sales']=merged_df['product_price']*merged_df['qty_sold']
monthly_product_sales_df=merged_df.groupby(['product_name','product_size','venue','year_month','year','month']).agg(
    monthly_sales=('sales','sum'),
    qty_sold_monthly=('qty_sold','sum'),
).reset_index()

avg_monthly_product_sales_df=monthly_product_sales_df.groupby(['product_name','product_size','month']).agg(
    avg_product_sales_by_month_of_the_year=('monthly_sales','mean')
).reset_index()

monthly_product_sales_df.head()
monthly_product_sales_df=pd.merge(merged_df,monthly_product_sales_df,how='right',on=['product_name','product_size','venue','year_month','year','month'])
monthly_product_sales_df=monthly_product_sales_df.drop(columns=['sales','qty_sold'])

monthly_product_sales_df=pd.merge(monthly_product_sales_df,avg_monthly_product_sales_df,how='left',on=['product_name','product_size','month'] )
monthly_product_sales_df['avg_product_sales_by_month_of_the_year']=monthly_product_sales_df['avg_product_sales_by_month_of_the_year'].apply(lambda x:round(x,2))

monthly_product_sales_df=monthly_product_sales_df.loc[:,['product_name','product_category','product_size','region',
                                'country','city','venue','year',
                                'year_month','month','product_price',
                                'qty_sold_monthly','monthly_sales','avg_product_sales_by_month_of_the_year']]

monthly_product_sales_df['monthly_sales']=monthly_product_sales_df['monthly_sales'].apply(lambda x:round(x,2))
monthly_product_sales_df=monthly_product_sales_df.drop_duplicates()
monthly_product_sales_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Sales_by_product.csv',index=False)



,region,country,city,venue,year,year_month,month,product_category,product_name,product_size,product_price,monthly_sales,qty_sold_monthly,avg_product_sales
0,LATAM,Argentina,Buenos Aires,Buenos Aires,2020,2020-03,03,food,butter croissant,one size,6.99,104.85,15,113.01
1,LATAM,Argentina,Buenos Aires,Buenos Aires,2020,2020-03,03,food,butter croissant,one size,6.99,104.85,15,113.01
2,LATAM,Argentina,Buenos Aires,Buenos Aires,2020,2020-03,03,food,butter croissant,one size,6.99,104.85,15,113.01
3,LATAM,Argentina,Buenos Aires,Buenos Aires,2020,2020-03,03,food,butter croissant,one size,6.99,104.85,15,113.01
4,LATAM,Argentina,Buenos Aires,Buenos Aires,2020,2020-03,03,food,butter croissant,one size,6.99,104.85,15,113.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21714,North America,USA,Seattle,Seattle,2025,2025-02,02,food,multigrain toast with butter,one size,5.99,83.86,14,52.76
21715,North America,USA,Seattle,Seattle,2025,2025-02,02,food,multigrain toast with butter,one size,5.99,83.86,14,52.76
21716,North America,USA,Seattle,Seattle,2025,2025-02,02,food,multigrain toast with butter,one size,5.99,83.86,14,52.76
21717,North America,USA,Seattle,Seattle,2025,2025-02,02,food,multigrain toast with butter,one size,5.99,83.86,14,52.76
